In [21]:
import numpy
import tflearn
import tensorflow as tf
import random
import json
import pickle
import nltk
import re
import string
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
stopwords = nltk.corpus.stopwords.words('english')



with open("intents_Sud.json") as file:
    data = json.load(file)

try:
    with open("data.pickle", "rb") as f:
        words, labels, training, output = pickle.load(f)
except:
    words = []
    labels = []
    docs_x = []
    docs_y = []

    for intent in data["intents"]:
        for pattern in intent["patterns"]:
            wrds = nltk.word_tokenize(pattern)
            #print("with tokenize:\n", wrds)
            wrds = [wrd for wrd in wrds if wrd.lower() not in stopwords]
            #print("without stopwords:\n", wrds)
            wrds = [wrd.lower() for wrd in wrds if wrd not in string.punctuation]
            #print("without punctuation:\n", wrds)
            words.extend(wrds)
            docs_x.append(wrds)
            docs_y.append(intent["tag"])

        if intent["tag"] not in labels:
            labels.append(intent["tag"])

    words = [lemmatizer.lemmatize(w.lower()) for w in words if w != "?"]
    words = sorted(list(set(words)))

    labels = sorted(labels)

    training = []
    output = []

    out_empty = [0 for _ in range(len(labels))]

    for x, doc in enumerate(docs_x):
        bag = []

        wrds = [lemmatizer.lemmatize(w.lower()) for w in doc]

        for w in words:
            if w in wrds:
                bag.append(1)
            else:
                bag.append(0)

        output_row = out_empty[:]
        output_row[labels.index(docs_y[x])] = 1

        training.append(bag)
        output.append(output_row)


    training = numpy.array(training)
    output = numpy.array(output)

    with open("data.pickle", "wb") as f:
        pickle.dump((words, labels, training, output), f)

tf.compat.v1.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)

#try:
#model.load('./model.tflearn')
#except:
model.fit(training, output, n_epoch=200, batch_size=8, show_metric=True)
model.save('./model.tflearn')



Training Step: 4799  | total loss: 0.25431 | time: 0.099s
| Adam | epoch: 200 | loss: 0.25431 - acc: 0.9471 -- iter: 184/189
Training Step: 4800  | total loss: 0.25434 | time: 0.104s
| Adam | epoch: 200 | loss: 0.25434 - acc: 0.9324 -- iter: 189/189
--
INFO:tensorflow:C:\Users\nitin\Python\Capstone\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [22]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [lemmatizer.lemmatize(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    return numpy.array(bag)


def chat():
    print("Start talking with the bot (type quit to stop)!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        results = model.predict([bag_of_words(inp, words)])
        results_index = numpy.argmax(results)
        tag = labels[results_index]
        #print(results)
        #print(tag)

        for tg in data["intents"]:
            if tg['tag'] == tag:
                responses = tg['responses']
            
        print("Bot: ",random.choice(responses))

chat()

Start talking with the bot (type quit to stop)!
You: hi
Bot:  Good to see you visit us.
You: hello
Bot:  Hi there, how can I help?
You: how to pay?
Bot:  COD is not available for now, we accept only online payments.
You: what is the address to visit?
Bot:  You can visit our shop at below address: \n V.L Mashere, shop no 4, halailohana mahajan trust building, chendani naka, Station Rd, next to patel saree center, near ashok cinema, Thane West, Thane, Maharashtra 400601
You: what are types of agarbattis you have?
Bot:  We have wide variety of puja related products and agarbattis.
You: which scented agarbattis you have?
Bot:  COD is not available for now, we accept only online payments.
You: do you have masala agarbattis?
Bot:  The costs of Wet Dhoop Sticks are as follows:Gugal-20pcs-35Rs,  Loban-20pcs-35Rs , Champa-20pcs-35Rs, Gulab-20pcs-35Rs, Mogra-20pcs-35Rs and Prachin-20pcs-35Rs. It will be packed in Zipper polybags.
You: premium masala
Bot:  We have Masala Agarbatti in different ty

In [9]:
labels

['DhoopPowder',
 'DhoopSticks',
 'DryDhoopSticks',
 'MasalaAgarbatti',
 'PremiumMasalaAgarbatti',
 'PremiumScentedAgarbatti',
 'ScentedAgarbatti',
 'WetDhoopSticks',
 'address',
 'goodbye',
 'greeting',
 'hours',
 'opentoday',
 'payments',
 'product',
 'quantity',
 'sell',
 'thanks']

In [40]:
stopwords

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [10]:
words

["'s",
 '50gm',
 'accept',
 'address',
 'agar',
 'agarbatti',
 'agarbattis',
 'akash',
 'akkalkot',
 'alphanso',
 'amchi',
 'anyone',
 'available',
 'azaroo',
 'blue',
 'box',
 'brut',
 'buy',
 'bye',
 'c',
 'card',
 'cash',
 'chafa',
 'champa',
 'chandan',
 'chintamani',
 'cod',
 'confidence',
 'cost',
 'credit',
 'dasang',
 'day',
 'delivery',
 'dhoop',
 'di',
 'different',
 'done',
 'dry',
 'english',
 'firaki',
 'flora',
 'flower',
 'get',
 'gold',
 'golden',
 'good',
 'goodbye',
 'gugal',
 'gulab',
 'heena',
 'hello',
 'helpful',
 'helping',
 'hi',
 'holy',
 'hour',
 'incense',
 'indrayani',
 'information',
 'jai',
 'jewel',
 'jordan',
 'kashmiri',
 'kasturi',
 'kesari',
 'king',
 'later',
 'lavender',
 'litchi',
 'loban',
 'lot',
 'magnum',
 'mallika',
 'mango',
 'mantra',
 'many',
 'maratha',
 'masala',
 'mastercard',
 'mata',
 'milan',
 'mogra',
 'much',
 'mumbai',
 'musk',
 'nabil',
 'nag',
 'namaste',
 'narayan',
 'natural',
 'nida',
 'nitya',
 'one',
 'open',
 'oudh',
 'pack

In [11]:
docs_x

[['hi'],
 [],
 ['anyone'],
 ['hello'],
 ['good', 'day'],
 ['wassup'],
 ['namaste'],
 ['bye'],
 ['see', 'later'],
 ['goodbye'],
 ['c', 'u', 'later'],
 ['tata'],
 ['bye'],
 ['thanks', 'lot'],
 ['thank'],
 ["'s", 'helpful'],
 ['thanks', 'information'],
 ['thank', 'helping'],
 ['hours', 'open'],
 ['hours'],
 ['open'],
 ['shop', 'open'],
 ['pay'],
 ['payments', 'done'],
 ['ways', 'payment'],
 ['accept', 'payment'],
 ['take', 'credit', 'cards'],
 ['accept', 'mastercard'],
 ['cash'],
 ['accept', 'cod'],
 ['cash', 'delivery'],
 ['open', 'today'],
 ['open', 'today'],
 ['hours', 'today'],
 ['sell'],
 ['u', 'sale'],
 ['selling'],
 ['available'],
 ['products', 'sold'],
 ['agarbattis'],
 ['sell', 'agarbattis'],
 ['agarbattis', 'available'],
 ['want', 'buy', 'agarbattis'],
 ['want', 'buy', 'incense', 'sticks'],
 ['shop', 'address'],
 ['shop'],
 ['shop'],
 ['visit', 'shop'],
 ['want', 'shop', 'address'],
 ['want', 'visit', 'shop'],
 ['visit', 'shop'],
 ['reach'],
 ['product'],
 ['product', 'box', 'pa

In [94]:
docs_y

['greeting',
 'greeting',
 'greeting',
 'greeting',
 'greeting',
 'greeting',
 'greeting',
 'goodbye',
 'goodbye',
 'goodbye',
 'goodbye',
 'goodbye',
 'goodbye',
 'thanks',
 'thanks',
 'thanks',
 'thanks',
 'thanks',
 'hours',
 'hours',
 'hours',
 'hours',
 'payments',
 'payments',
 'payments',
 'payments',
 'payments',
 'payments',
 'payments',
 'payments',
 'payments',
 'opentoday',
 'opentoday',
 'opentoday',
 'sell',
 'sell',
 'sell',
 'sell',
 'sell',
 'sell',
 'sell',
 'sell',
 'sell',
 'sell',
 'address',
 'address',
 'address']

In [34]:
training[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0])

In [35]:
output[0]

array([0, 1, 0, 0, 0, 0, 0])